<a href="https://colab.research.google.com/github/Julia2505/PDD/blob/master/Train_Siamese_Network_(02_12_2019).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Siamese Network for PDD data

In this example we will show, how to train your own classifier using [Plant Disease Database](http://pdd.jinr.ru/crops.php)

### Cloning the repo

At first we will clone the repository.

In [0]:
!rm -r -f pdd_new
!git clone https://github.com/AlexanderUzhinskiy/pdd_new.git

Cloning into 'pdd_new'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 278 (delta 0), reused 0 (delta 0), pack-reused 273
Receiving objects: 100% (278/278), 53.90 MiB | 31.65 MiB/s, done.
Resolving deltas: 100% (165/165), done.


Change the directory to pdd_new to get access of helper functions and classes. Check github for mode details.

In [0]:
import os
os.chdir('pdd_new')
# verify if we are in correct directory
os.listdir()

['README.md', '.gitignore', 'pdd', '.git', 'server', 'examples']

### Downloading the grape pdd dataset 

We load it and split on train and test subdirectories in place. Chenge path and origin to your database. For sample of the structure download - http://pdd.jinr.ru/crops_nn.tar

In [0]:
from pdd.datasets.grape import load_data

train_data_path, test_data_path = load_data(path='crops_nn.tar', origin="http://pdd.jinr.ru/crops_nn.tar", split_on_train_test=True, random_state=13)
print(train_data_path)

Using TensorFlow backend.


100%|███████████████████| 15/15 [00:00<00:00, 121.34it/s]

Splitting on train and test...
/tmp/.pdd/datasets/crops_nn/train


The structure of the dataset catalogue now:

```
grape
│    
└───train
│   └───crop_disease_name
│   |   │   20.jpg
│   |   │   ...
|   |
|   └───crop_disease_name
|   └───crop_disease_name
|   └───crop_disease_name
|   └───crop_disease_name
|
└───test
    └───crop_disease_name
    |   │   3.jpg
    |   │   ...
    |
    └───crop_disease_name
    └───crop_disease_name
    └───crop_disease_name
    └───crop_disease_name
```

### Create a feature extractor `twin` and a siamese network

In [0]:
from pdd.models import get_feature_extractor
from pdd.models import make_siamese
import keras.backend as K
import tensorflow as tf

# set the single session for tensorflow and keras both
sess = tf.Session()
K.set_session(sess)

We are using the cross-entropy loss as in [this paper](http://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf) instead of contrastive loss. 

But feel free for using it by changing parameter `loss` to 'contrastive'. E.g.

```python
siams = make_siamese(feature_extractor, dist='l2', loss='contrastive')
```

There are three types of distances:


*   `'l1'`
*   `'l2'`
*   `'cosine'`

**But only `'l1'` is available for cross-entropy loss.**



In [0]:
input_shape = (256, 256, 3)

print("Building feature extractor...")
feature_extractor = get_feature_extractor(input_shape)

print("Constructing siamese network...")
siams = make_siamese(feature_extractor, dist='l1', loss='cross_entropy')
siams.summary()

Building feature extractor...










Constructing siamese network...


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 1024)         14902496    input_2[0][0]                    
                                                                 input_3[0][0]                    
_______________________

### Creating a batch generator

To train a siamese network data should be passed to the input as **positive-negative pairs**. Positive pair of images means a pair consists of images from the same class. Negative pairs consist of images of different classes.

In [0]:
from pdd.utils.training import SiameseBatchGenerator

For training we are using a strong augmentation including rotations, zooming, flips and channel shifts.

In [0]:
train_batch_gen = SiameseBatchGenerator.from_directory(dirname=train_data_path, augment=True)
test_batch_gen = SiameseBatchGenerator.from_directory(dirname=test_data_path)

print(train_batch_gen) 

def siams_generator(batch_gen, batch_size=None):
    while True:
        batch_xs, batch_ys = batch_gen.next_batch(batch_size)
        yield [batch_xs[0], batch_xs[1]], batch_ys

### Training

Let's train the model. 

**Note**, despite the fact that we are training the siamese model, the feature extractor is also being trained.

In [0]:
siams.fit_generator(
    generator=siams_generator(train_batch_gen),
    steps_per_epoch=100,
    epochs=100,
    verbose=1,
    validation_data=siams_generator(test_batch_gen),
    validation_steps=30,
    shuffle=True
)

Epoch 1/100
100/100 [==============================] - 142s 1s/step - loss: 0.0920 - acc: 0.9725 - val_loss: 0.5044 - val_acc: 0.8042
Epoch 2/100
100/100 [==============================] - 136s 1s/step - loss: 0.0933 - acc: 0.9688 - val_loss: 0.6036 - val_acc: 0.7458
Epoch 3/100
100/100 [==============================] - 137s 1s/step - loss: 0.0863 - acc: 0.9712 - val_loss: 0.4961 - val_acc: 0.7812
Epoch 4/100
100/100 [==============================] - 136s 1s/step - loss: 0.0928 - acc: 0.9659 - val_loss: 0.4921 - val_acc: 0.8042
Epoch 5/100
100/100 [==============================] - 136s 1s/step - loss: 0.0948 - acc: 0.9684 - val_loss: 0.5197 - val_acc: 0.8177
Epoch 6/100
100/100 [==============================] - 136s 1s/step - loss: 0.0801 - acc: 0.9728 - val_loss: 0.4240 - val_acc: 0.8250
Epoch 7/100
100/100 [==============================] - 135s 1s/step - loss: 0.0804 - acc: 0.9728 - val_loss: 0.4510 - val_acc: 0.8333
Epoch 8/100
100/100 [==============================] - 135s 1s

KeyboardInterrupt: ignored

### K-nearest neighbors for classification

Using of a siamese network for classification requires to perform an iterative n-shot task. But to avoid this restriction we can build a KNN classifier with the help of features extracted using the `twin` network.

To significantly speed up the inference phase of the classifer we are going to utilize a fast k-nearest-neighbour search based on the method used in ["Learning To Remember Rare Events"](https://openreview.net/pdf?id=SJTQLdqlg).

#### Save the feature extractor model and clear session

In [0]:
print("Saving feature extractor...")
feature_extractor.save('pdd_feature_extractor.h5')

Saving feature extractor...


In [0]:
K.clear_session()
tf.reset_default_graph()
del sess

#### Load the feature exctrator to KNN model

In [0]:
from sklearn.metrics import accuracy_score
from keras.models import load_model

from pdd.models import TfKNN
from pdd.utils.data_utils import create_dataset_from_dir

In [0]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

print("Loading feature extractor...")
feature_extractor = load_model("pdd_feature_extractor.h5")

Loading feature extractor...


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


#### Load datasets for the evaluation

In [0]:
print("Loading datasets...")
train_dataset = create_dataset_from_dir(train_data_path, shuffle=True)
test_dataset = create_dataset_from_dir(test_data_path, shuffle=True)

Loading datasets...


#### Create KNN graph

In [0]:
tfknn = TfKNN(sess, 
              feature_extractor, 
              (train_dataset['data'], train_dataset['target']))

Getting keys from support set...
--[_get_keys_from_support_set] took 1.10 seconds to run.

Freezing feature extractor graph...
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 52 variables.
INFO:tensorflow:Converted 52 variables to const ops.



--[_freeze_feature_extractor_graph] took 0.42 seconds to run.

Creating TfKNN graph...
--[make_fknn_graph] took 0.02 seconds to run.



#### Evaluate

In [0]:
# predictions and similarities
preds, sims = tfknn.predict(test_dataset['data'])
accuracy = accuracy_score(test_dataset['target'], preds)
print("Accuracy: %.2f" % accuracy)


Making prediction for 70 images...
--[predict] took 0.93 seconds to run.

Accuracy: 0.80


#### Save the model for tensorflow serving

In [0]:
tfknn.save_graph_for_serving("tfknn_graph")

Saving graph for serving...
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: tfknn_graph/saved_model.pb
--[save_graph_for_serving] took 0.20 seconds to run.



*italicized text*## Upload model to Google Drive

Mount your Google Drive

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Copy files to gdrive

In [0]:
# make directory in the destination
!mkdir "/gdrive/My Drive/pdd_model_0612"

 #copy models .
!cp tfknn_graph/saved_model.pb "/gdrive/My Drive/pdd_model_0612/tf_graph.pb"
!cp pdd_feature_extractor.h5 "/gdrive/My Drive/pdd_model_0612/pdd_feature_extractor.h5"